<a href="https://colab.research.google.com/github/acb-code/example-agents/blob/main/vision_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vision Agents with smolagents


This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

## Let's install the dependencies and login to our HF account to access the Inference API

If you haven't installed `smolagents` yet, you can do so by running the following command:

In [1]:
# !pip install smolagents
!pip install 'smolagents[litellm]' google-genai -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.5 MB/s eta 0:00:00
  Attempting uninstall: anyio
    Found existing installation: anyio 3.7.1
    Uninstalling anyio-3.7.1:
      Successfully uninstalled anyio-3.7.1
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: google-genai
    Found existing installation: go

Let's also login to the Hugging Face Hub to have access to the Inference API.

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
# testing access to gemini
from google import genai
from google.colab import userdata
gemini_api_key = userdata.get('GOOGLE_API_KEY')

client = genai.Client(api_key=gemini_api_key)
response = client.models.generate_content(
    model="gemini-2.0-flash-lite", contents="Explain how AI works in 10 words or less"
)
print(response.text)

Algorithms analyze data, learn, and make predictions/decisions.



In [3]:
# setting up model for smolagents to use
from smolagents import LiteLLMModel

gemini_model = LiteLLMModel(model_id="gemini/gemini-2.0-flash-lite", api_key=gemini_api_key)

## Providing Images at the Start of the Agent's Execution

In this approach, images are passed to the agent at the start and stored as `task_images` alongside the task prompt. The agent then processes these images throughout its execution.  

Consider the case where Alfred wants to verify the identities of the superheroes attending the party. He already has a dataset of images from previous parties with the names of the guests. Given a new visitor's image, the agent can compare it with the existing dataset and make a decision about letting them in.  

In this case, a guest is trying to enter, and Alfred suspects that this visitor might be The Joker impersonating Wonder Woman. Alfred needs to verify their identity to prevent anyone unwanted from entering.  

Let’s build the example. First, the images are loaded. In this case, we use images from Wikipedia to keep the example minimal, but image the possible use-case!

In [6]:
from PIL import Image
import requests
from io import BytesIO
import time

image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/e/e8/The_Joker_at_Wax_Museum_Plus.jpg",
    "https://upload.wikimedia.org/wikipedia/en/9/98/Joker_%28DC_Comics_character%29.jpg"
]

images = []
for url in image_urls:
    time.sleep(1)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    # response = requests.get(url)
    response = requests.get(url, headers=headers, stream=True)
    response.raise_for_status()  # Check for any HTTP errors
    image = Image.open(BytesIO(response.content)).convert("RGB")
    images.append(image)

Now that we have the images, the agent will tell us wether the guests is actually a superhero (Wonder Woman) or a villian (The Joker).

In [ ]:
# from google.colab import userdata
# import os
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# using gemini flash lite instead

In [7]:
from smolagents import CodeAgent, OpenAIServerModel

# model = OpenAIServerModel(model_id="gpt-4o")
model = gemini_model

# Instantiate the agent
agent = CodeAgent(
    tools=[],
    model=model,
    max_steps=20,
    verbosity_level=2
)

response = agent.run(
    """
    Describe the costume and makeup that the comic character in these photos is wearing and return the description.
    Tell me if the guest is The Joker or Wonder Woman.
    """,
    images=images
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Describe the costume and makeup that the comic character in these photos is wearing and return the description. │
│     Tell me if the guest is The Joker or Wonder Woman.                                                          │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash-lite ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: Based on the images, I can describe the costume and makeup and then identify the character.               
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
final_answer("The character in the photos is wearing white face makeup, green hair, and a large, exaggerated smile.
The costume consists of a purple overcoat, a yellow shirt, and a purple ascot. The character is The Joker.")       
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The character in the photos is wearing white face makeup, green hair, and a large, exaggerated     
  smile. The costume consists of a purple overcoat, a yellow shirt, and a purple ascot. The character is The       
  Joker.")                                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The character in the photos is wearing white face makeup, green hair, and a large, exaggerated 
smile. The costume consists of a purple overcoat, a yellow shirt, and a purple ascot. The character is The Joker.

[Step 1: Duration 8.16 seconds| Input tokens: 2,721 | Output tokens: 78]

In [8]:
response

'The character in the photos is wearing white face makeup, green hair, and a large, exaggerated smile. The costume consists of a purple overcoat, a yellow shirt, and a purple ascot. The character is The Joker.'

In this case, the output reveals that the person is impersonating someone else, so we can prevent The Joker from entering the party!

## Providing Images with Dynamic Retrieval

This examples is provided as a `.py` file since it needs to be run locally since it'll browse the web. Go to the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course) for more details.